In [1]:
import sys
import re
import time
import os
import requests
sys.path.append("..")

from rich.console import Console
from rich.progress import Progress
from twocaptcha import TwoCaptcha

from datetime import datetime
from tempfile import NamedTemporaryFile

from models.cases import Case
from models.leads import Lead
from models.scraper import ScraperBase

from dotenv import load_dotenv
load_dotenv(dotenv_path='.env')
TWOCAPTCHA_API_KEY = os.getenv('TWOCAPTCHA_API_KEY')

console = Console()

class IndianaScraper(ScraperBase):
    solver = TwoCaptcha(TWOCAPTCHA_API_KEY)
    field_mapping = {
        "CaseNumber": "case_id",
        "Court": "court_desc",
        "CaseType": "case_type",
        "CaseStatus": "case_status",
        "FileDate":"filing_date",
        "AppearByDate": "court_date",
        "Style": "description",
    }

    def split_full_name(self, name):
        # Use regular expression to split on space, comma, hyphen, or period.
        # This can be expanded to include other delimiters if required.
        parts = re.split(r'[\s,\-\.]+', name)
        
        # Prepare variables for first, middle, and last names
        first_name = middle_name = last_name = ''

        # The list 'parts' now contains the split name parts.
        # How we assign these parts depends on the number of elements in 'parts'.
        if len(parts) > 2:
            first_name = parts[0]
            middle_name = ' '.join(parts[1:-1])  # All parts except first and last are considered middle names
            last_name = parts[-1]
        elif len(parts) == 2:
            first_name, last_name = parts
        elif len(parts) == 1:
            first_name = parts[0]

        return first_name, middle_name, last_name
    
    def get_case_detail(self, case_token):
        url = "https://public.courts.in.gov/mycase/Case/CaseSummary"
        params = {
            'SRCT': '',
            'CaseToken': case_token,
            '_': '1713862566949'
        }
        headers = {
            'Content-Type': 'application/json',
            'X-Requested-With': 'XMLHttpRequest'
        }
        res = requests.get(url, params=params, headers=headers, timeout=10)
        if res.status_code == 200:
            return res.json()
        else:
            return None
    
    def extract_case_info(self, case_detail):
        case_dict = {
            value: case_detail.get(key) for key, value in self.field_mapping.items()
        }
        case_dict["filing_date"] = datetime.strptime(case_dict["filing_date"], "%m/%d/%Y") if case_dict["filing_date"] else None
        case_dict["court_date"] = datetime.strptime(case_dict["court_date"], "%m/%d/%Y") if case_dict["court_date"] else None
        
        court_id = case_detail.get("CountyCode")+case_detail.get("CourtCode")
        charges = [
            {
                "offense_date": charge.get("OffenseDate"),
                "description": charge.get("OffenseDescription"),
                "degree": charge.get("OffenseDegree"),
            }
            for charge in case_detail.get("Charges", [])
        ]
        if charges:
            offense_date = charges[0].get("offense_date")
            offense_date = datetime.strptime(offense_date, "%m/%d/%Y") if offense_date else None
        else:
            offense_date = None

        events = [
            {
                "event_date": event.get("EventDate"),
                "event_description": event.get("Description"),
                "event_type": event.get("EventType"),
                "judge": event.get("Judge"),
                "comment": event.get("CaseEvent").get("Comment") if event.get("CaseEvent") else None
            }
            for event in case_detail.get("Events", [])
        ]
        parties = [
            {
                "role": party.get("ExtConnCodeDesc"),
                "name": party.get("Name"),
            }
            for party in case_detail.get("Parties", [])
        ]
        defendant = [party for party in case_detail.get("Parties") if party.get("ExtConnCode") == "DEF"]
        if len(defendant) == 0:
            return {
                "court_id": court_id,
                "charges": charges,
                "offense_date": offense_date,
                "events": events,
                "parties": parties,
                **case_dict,
            }
        else:
            defendant = defendant[0]
        first_name, middle_name, last_name = self.split_full_name(defendant.get("Name"))
        birth_date = defendant.get("DOB")
        address_line_1 = defendant.get("Address").get("Line1") if defendant.get("Address") else None
        address_city = defendant.get("Address").get("City") if defendant.get("Address") else None
        address_state_code = defendant.get("Address").get("State") if defendant.get("Address") else None
        address_zip = defendant.get("Address").get("Zip") if defendant.get("Address") else None

        return {
            "court_id": court_id,
            "charges": charges,
            "offense_date": offense_date,
            "events": events,
            "parties": parties,
            "first_name": first_name,
            "middle_name": middle_name,
            "last_name": last_name,
            "birth_date": birth_date,
            "address_line_1": address_line_1,
            "address_city": address_city,
            "address_state_code": address_state_code,
            "address_zip": address_zip,
            **case_dict,
        }
    
    def get_cases(self, filed_date, captcha=None):
        url = "https://public.courts.in.gov/mycase/Search/SearchCases"
        captcha_answer = {"Key":captcha.get("key"),"Answer":captcha.get("answer"),"Refresh":False} if captcha else None
        data = {
            "Mode": "ByCase", 
            "Categories": ["CR"],
            "CaptchaAnswer": captcha_answer,
            "Advanced": True,
            "ActiveFlag": "Open",
            "FileStart": filed_date,
            "FileEnd": filed_date,
            "Skip": 0,
            "Take": 9999,
            "Sort": "FileDate DESC"
        }
        headers = {
            'Content-Type': 'application/json',
            'X-Requested-With': 'XMLHttpRequest'
        }
        res = requests.post(url, json=data, headers=headers)
        
        if res.status_code != 200:
            return []
        elif res.json().get("Results") is None:
            img = res.json().get('Url')
            key = res.json().get('CaptchaKey')
            answer = self.solver.normal(img)["code"]
            return self.get_cases(filed_date, {"key":key,"answer":answer})
        else:
            return [case.get("CaseToken") for case in res.json().get("Results")]
        
    def scrape(self, search_parameters):
        filed_date = search_parameters['filed_date']
        case_tokens = self.get_cases(filed_date)
        console.log(f"Found {len(case_tokens)} cases")
        with Progress() as progress:
            task = progress.add_task("[red]Inserting cases...", total=len(case_tokens))
            for case_token in case_tokens[:3]:
                case_detail = self.get_case_detail(case_token)
                if case_detail:
                    case_dict = self.extract_case_info(case_detail)
                    print(case_dict)
                    case = Case(**case_dict)
                    lead = Lead(**case_dict)
                    self.insert_case(case)
                    self.insert_lead(lead)
                else:
                    console.log(case_token)
                time.sleep(10)  # Add additional sleep time to avoid get blocked
                progress.update(task, advance=1)

In [2]:
indiana_scraper = IndianaScraper()
indiana_scraper.scrape({'filed_date': '04/12/2024'})

[08:41:10] Found 596 cases                                                                        ]8;id=247248;file:///tmp/ipykernel_7878/1916727690.py\1916727690.py]8;;\:]8;id=818797;file:///tmp/ipykernel_7878/1916727690.py#185\185]8;;\

/home/whitesnowman113/Work/scrapers/venv/lib/python3.11/site-packages/rich/live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

{'court_id': '30D02', 'charges': [{'offense_date': '04/08/2024', 'description': "9-30-4-8/MC: Operating Vehicle 
with Suspended Registration Def.'s vehicle registration is suspended", 'degree': 'MC'}], 'offense_date': 
datetime.datetime(2024, 4, 8, 0, 0), 'events': [{'event_date': '04/12/2024', 'event_description': 'Case Opened as a
New Filing', 'event_type': 'QCSNEW', 'judge': None, 'comment': None}, {'event_date': '04/12/2024', 
'event_description': 'Case Filed Electronically', 'event_type': 'CFE', 'judge': None, 'comment': 'Added By EFile 
Manager'}, {'event_date': '04/12/2024', 'event_description': 'Information Filed', 'event_type': 'PCRINF', 'judge': 
None, 'comment': 'Information'}, {'event_date': '04/12/2024', 'event_description': 'Probable Cause Affidavit 
Filed', 'event_type': 'ADMPCAFF', 'judge': None, 'comment': 'ERIC ORIA REDACTED PC.pdf'}, {'event_date': 
'04/13/2024', 'event_description': 'Automated Paper Notice Issued to Parties', 'event_type': 'NITPB', 'judge': 
None, 'comment': 'Probable Cause Affidavit Filed ---- 4/12/2024 : Eric Jaime Oria'}, {'event_date': '04/13/2024', 
'event_description': 'Automated ENotice Issued to Parties', 'event_type': 'ENITPB', 'judge': None, 'comment': 
'Probable Cause Affidavit Filed ---- 4/12/2024 : Brent Edward Eaton'}, {'event_date': '04/16/2024', 
'event_description': 'Probable Cause Found: Order Issued', 'event_type': 'ARJPCF', 'judge': 'Coombs, Cody B. - 
MAG', 'comment': "COURT FINDS PROBABLE CAUSE. DEFENDANT IS ORDERED TO APPEAR FOR INITIAL HEARING ON MAY 8, 2024 AT 
8:30 A.M. FAILURE TO APPEAR WILL RESULT IN A WARRANT FOR THE DEFENDANT'S ARREST. TAH"}, {'event_date': 
'04/16/2024', 'event_description': 'Hearing Scheduling Activity', 'event_type': 'HSACT', 'judge': None, 'comment': 
'Initial Hearing scheduled for 05/08/2024 at 8:30 AM.'}, {'event_date': '04/17/2024', 'event_description': 
'Automated Paper Notice Issued to Parties', 'event_type': 'NITPB', 'judge': None, 'comment': 'Probable Cause Found:
Order Issued ---- 4/16/2024 : Eric Jaime Oria\nHearing Scheduling Activity ---- 4/16/2024 : Eric Jaime Oria'}, 
{'event_date': '04/17/2024', 'event_description': 'Automated ENotice Issued to Parties', 'event_type': 'ENITPB', 
'judge': None, 'comment': 'Hearing Scheduling Activity ---- 4/16/2024 : Brent Edward Eaton'}, {'event_date': 
'04/17/2024', 'event_description': 'Appearance Filed', 'event_type': 'APP', 'judge': None, 'comment': 
'Appearance'}, {'event_date': '05/08/2024', 'event_description': 'Initial Hearing', 'event_type': 'HIH', 'judge': 
'Marshall, Dan E', 'comment': None}, {'event_date': '05/08/2024', 'event_description': 'Hearing Journal Entry', 
'event_type': 'HJE', 'judge': 'Marshall, Dan E', 'comment': "DEFENDANT FAILED TO APPEAR FOR INITIAL HEARING. CLERK 
IS ORDERED TO ISSUE A WARRANT FOR THE DEFENDANT'S ARREST; HELD WITHOUT BOND PENDING APPEARANCE BEFORE THIS COURT. 
TAH"}, {'event_date': '05/08/2024', 'event_description': 'Warrant or Writ of Attmnt for the Body of a Person 
Issued', 'event_type': 'WI', 'judge': None, 'comment': None}, {'event_date': '05/08/2024', 'event_description': 
'Service Issued', 'event_type': 'SI', 'judge': None, 'comment': 'FTA WARRANT SCANNED TO HCSD. WR'}, {'event_date': 
'05/09/2024', 'event_description': 'Automated Paper Notice Issued to Parties', 'event_type': 'NITPB', 'judge': 
None, 'comment': 'Hearing Journal Entry ---- 5/8/2024 : Eric Jaime Oria'}], 'parties': [{'role': 'Defendant', 
'name': 'Oria, Eric Jaime'}, {'role': 'State Plaintiff', 'name': 'State of Indiana'}], 'first_name': 'Oria', 
'middle_name': 'Eric', 'last_name': 'Jaime', 'birth_date': None, 'address_line_1': '9431 Verbena Drive', 
'address_city': 'Munster', 'address_state_code': 'IN', 'address_zip': '46321', 'case_id': '30D02-2404-CM-000658', 
'court_desc': 'Hancock Superior Court 2', 'case_type': 'CM - Criminal Misdemeanor', 'case_status': 'Pending', 
'filing_date': datetime.datetime(2024, 4, 12, 0, 0), 'court_date': None, 'description': 'State of Indiana 

{'court_id': '45D07', 'charges': [{'offense_date': '04/08/2024', 'description': "9-24-1-1(a)/IFC: No Valid Driver's
License", 'degree': 'IFC'}], 'offense_date': datetime.datetime(2024, 4, 8, 0, 0), 'events': [{'event_date': 
'04/12/2024', 'event_description': 'Case Opened as a New Filing', 'event_type': 'QCSNEW', 'judge': None, 'comment':
None}, {'event_date': '04/12/2024', 'event_description': 'Traffic Citation Filed', 'event_type': 'PCRTCT', 'judge':
None, 'comment': None}, {'event_date': '05/01/2024', 'event_description': 'Hearing Scheduling Activity', 
'event_type': 'HSACT', 'judge': None, 'comment': 'Initial Hearing scheduled for 12/17/2024 at 1:00 PM.'}, 
{'event_date': '05/02/2024', 'event_description': 'Automated Paper Notice Issued to Parties', 'event_type': 
'NITPB', 'judge': None, 'comment': 'Hearing Scheduling Activity ---- 5/1/2024 : ALIANA D JEFFERSON'}, 
{'event_date': '05/02/2024', 'event_description': 'Automated ENotice Issued to Parties', 'event_type': 'ENITPB', 
'judge': None, 'comment': 'Hearing Scheduling Activity ---- 5/1/2024 : Bernard Arnold Carter'}, {'event_date': 
'12/17/2024', 'event_description': 'Initial Hearing', 'event_type': 'HIH', 'judge': None, 'comment': None}], 
'parties': [{'role': 'Defendant', 'name': 'JEFFERSON, ALIANA D'}, {'role': 'State Plaintiff', 'name': 'State of 
Indiana'}], 'first_name': 'JEFFERSON', 'middle_name': 'ALIANA', 'last_name': 'D', 'birth_date': None, 
'address_line_1': '832 EATON STREET', 'address_city': 'HAMMOND', 'address_state_code': 'IN', 'address_zip': 
'46320', 'case_id': '45D07-2404-IF-006845', 'court_desc': 'Lake Superior Court, County Division 1', 'case_type': 
'IF - Infraction', 'case_status': 'Pending', 'filing_date': datetime.datetime(2024, 4, 12, 0, 0), 'court_date': 
datetime.datetime(2024, 6, 7, 0, 0), 'description': 'State of Indiana v. ALIANA D JEFFERSON'}

{'court_id': '45D07', 'charges': [{'offense_date': '03/31/2024', 'description': '9-30-5-2(a) & (b)/MA: Operating a 
Vehicle While Intoxicated Endangering a Person', 'degree': 'MA'}, {'offense_date': '03/31/2024', 'description': 
'9-26-1-1.1(a)(1)/MB: Leaving the Scene of an Accident Driver fails to immediately stop at the scene', 'degree': 
'MB'}, {'offense_date': '03/31/2024', 'description': '9-30-5-2(a)/MC: Operating a Vehicle While Intoxicated-first 
offense and no endangerment', 'degree': 'MC'}, {'offense_date': '03/31/2024', 'description': 
'9-18.1-11-2(b)(2)/IFC: Operating with Expired Plates', 'degree': 'IFC'}, {'offense_date': '03/31/2024', 
'description': '9-24-19-1/IFA: Driving While Suspended', 'degree': 'IFA'}], 'offense_date': datetime.datetime(2024,
3, 31, 0, 0), 'events': [{'event_date': '04/12/2024', 'event_description': 'Case Opened as a New Filing', 
'event_type': 'QCSNEW', 'judge': None, 'comment': None}, {'event_date': '04/12/2024', 'event_description': 'Case 
Filed Electronically', 'event_type': 'CFE', 'judge': None, 'comment': 'Added By EFile Manager'}, {'event_date': 
'04/12/2024', 'event_description': 'Appearance Filed', 'event_type': 'APP', 'judge': None, 'comment': 
'Appearance'}, {'event_date': '04/12/2024', 'event_description': 'Traffic Citation Filed', 'event_type': 'PCRTCT', 
'judge': None, 'comment': 'TICKET.pdf'}, {'event_date': '04/12/2024', 'event_description': 'Information Filed', 
'event_type': 'PCRINF', 'judge': None, 'comment': 'INFO.pdf'}, {'event_date': '04/12/2024', 'event_description': 
'Probable Cause Affidavit Filed', 'event_type': 'ADMPCAFF', 'judge': None, 'comment': 'PC.pdf'}, {'event_date': 
'04/12/2024', 'event_description': 'Probable Cause Found: Order Issued', 'event_type': 'ARJPCF', 'judge': None, 
'comment': None}, {'event_date': '04/13/2024', 'event_description': 'Automated ENotice Issued to Parties', 
'event_type': 'ENITPB', 'judge': None, 'comment': 'Probable Cause Found: Order Issued ---- 4/12/2024 : Bernard 
Arnold Carter'}, {'event_date': '04/13/2024', 'event_description': 'Automated Paper Notice Issued to Parties', 
'event_type': 'NITPB', 'judge': None, 'comment': 'Probable Cause Found: Order Issued ---- 4/12/2024 : CAMERON M 
LLOYD'}, {'event_date': '04/15/2024', 'event_description': 'Order to Appear', 'event_type': 'OTA', 'judge': None, 
'comment': None}, {'event_date': '04/15/2024', 'event_description': 'Hearing Scheduling Activity', 'event_type': 
'HSACT', 'judge': None, 'comment': 'Initial Hearing scheduled for 04/29/2024 at 1:00 PM.'}, {'event_date': 
'04/16/2024', 'event_description': 'Automated Paper Notice Issued to Parties', 'event_type': 'NITPB', 'judge': 
None, 'comment': 'Order to Appear ---- 4/15/2024 : CAMERON M LLOYD\nHearing Scheduling Activity ---- 4/15/2024 : 
CAMERON M LLOYD'}, {'event_date': '04/16/2024', 'event_description': 'Automated ENotice Issued to Parties', 
'event_type': 'ENITPB', 'judge': None, 'comment': 'Order to Appear ---- 4/15/2024 : Bernard Arnold Carter\nHearing 
Scheduling Activity ---- 4/15/2024 : Bernard Arnold Carter'}, {'event_date': '04/29/2024', 'event_description': 
'Initial Hearing', 'event_type': 'HIH', 'judge': None, 'comment': None}, {'event_date': '04/29/2024', 
'event_description': 'Failure to Appear', 'event_type': 'ADMFTA', 'judge': None, 'comment': None}, {'event_date': 
'04/29/2024', 'event_description': 'Order Issuing Warrant for Arrest', 'event_type': 'OIWA', 'judge': None, 
'comment': 'Court Sets Bond $3000'}, {'event_date': '04/29/2024', 'event_description': 'Probable Cause: Refused 
with Immediate Suspension', 'event_type': 'BMVPCRSUSP', 'judge': None, 'comment': None}, {'event_date': 
'04/29/2024', 'event_description': 'PCZNT - An Electronic PC Affidavit was created in INcite', 'event_type': 
'BMVPCZNT', 'judge': None, 'comment': 'Probable cause affidavit sent to BMV'}, {'event_date': '04/30/2024', 
'event_description': 'Automated Paper Notice Issued to Parties', 'event_type': 'NITPB', 'judge': None, 'commen